# A Full Business Solution for a Investment Management Company

### BUSINESS CHALLENGE:

Create a product that builds a 1 Page Pitch for an investment company to convince them to purchase a stock and include it in their portfolio.


In [1]:
#!pip install markdown

In [2]:
# imports

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [4]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
I will use "one shot prompting" in which I provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

In [5]:
link_system_prompt = """
You are a link extraction agent. Your task is to extract only the most relevant pages from a company's website that would support writing a professional 1-page investment pitch for institutional investors.

You must return only a valid JSON object. Do not include any commentary or explanation. Your output must include:
- The company homepage as "homepage"
- A list of relevant links as "links", with each link having:
  - "type": a category (e.g., about page, investor relations page, products or services page, etc.)
  - "url": the full URL

Relevant page types include:
- about page
- company overview page
- investor relations page
- products or services page
- esg or sustainability page
- leadership or team page

---

Example 1:
Input:
[
  "https://acme.com",
  "https://acme.com/about",
  "https://acme.com/investor-relations",
  "https://acme.com/contact"
]

Output:
{
  "homepage": "https://acme.com",
  "links": [
    {"type": "about page", "url": "https://acme.com/about"},
    {"type": "investor relations page", "url": "https://acme.com/investor-relations"}
  ]
}

---

Example 2:
Input:
[
  "https://futureenergy.com",
  "https://futureenergy.com/about-us",
  "https://futureenergy.com/investors",
  "https://futureenergy.com/solutions",
  "https://futureenergy.com/blog"
]

Output:
{
  "homepage": "https://futureenergy.com",
  "links": [
    {"type": "about page", "url": "https://futureenergy.com/about-us"},
    {"type": "investor relations page", "url": "https://futureenergy.com/investors"},
    {"type": "products or services page", "url": "https://futureenergy.com/solutions"}
  ]
}

Now return the JSON output for the following input:
"""


In [6]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a pitch 1 pager about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [7]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


## Second Step: Make the Pitch Deck!

Assemble all the details into another prompt to GPT4-o

In [8]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [9]:
system_prompt = """
You are a senior investment analyst assistant tasked with creating a vibrant, high-quality, and professional 1-page stock pitch in Markdown format.

This pitch is for institutional investors deciding whether to BUY the stock.

Your response must:
- Be written in **Markdown**
- Include the **company logo** rendered using Clearbit, but only once, directly under the “Company Name and Website” section
- Be structured, scannable, and visually appealing using tasteful emojis and bold headers
- End with a clear, data-supported investment recommendation

### Recommended Sections:
- 📌 **Company Name and Website**
- ![Company Logo](https://logo.clearbit.com/{domain}) ← Place here
- 🏢 **Company Overview**
- ⚙️ **Products & Services**
- 💼 **Business Model**
- 📈 **Growth Strategy**
- 🌱 **ESG & Sustainability** (if relevant)
- 👥 **Leadership**
- 💰 **Financial & Investor Highlights**
- 🧠 **Competitive Edge**
- ✅ **Final Recommendation**

Keep the pitch concise, punchy, and under one page. Use real business reasoning, and end with a firm BUY/HOLD/SELL call.
Output only the final Markdown.
"""

In [10]:
def get_brochure_user_prompt(company_name, homepage_url, domain, relevant_links, page_contents=None):
    user_prompt = f"""
Here is the company data:

- Company Name: {company_name}
- Website: {homepage_url}
- Domain for logo: {domain}
- Relevant pages:
{json.dumps(relevant_links, indent=2)}
"""

    if page_contents:
        user_prompt += f"\n\n### Page Contents:\n{page_contents[:20000]}"

    user_prompt += "\n\nPlease use this to write a 1-page Markdown pitch for institutional investors."

    return user_prompt


In [11]:
from urllib.parse import urlparse

def create_brochure(company_name, homepage_url, relevant_links):
    domain = urlparse(homepage_url).netloc

    user_prompt = get_brochure_user_prompt(
        company_name=company_name,
        homepage_url=homepage_url,
        domain=domain,
        relevant_links=relevant_links
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    result = response.choices[0].message.content
    display(Markdown(result))


## Finally - A Minor Improvement

With a small adjustment, I can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [12]:
from urllib.parse import urlparse
from markdown import markdown as md_to_html
from IPython.display import Markdown, display, update_display

def stream_brochure(company_name, homepage_url, relevant_links, save_as_html=True):
    domain = urlparse(homepage_url).netloc

    user_prompt = get_brochure_user_prompt(
        company_name=company_name,
        homepage_url=homepage_url,
        domain=domain,
        relevant_links=relevant_links
    )

    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        new_text = chunk.choices[0].delta.content or ""
        response += new_text
        cleaned = response.replace("```", "").replace("markdown", "")
        update_display(Markdown(cleaned), display_id=display_handle.display_id)

    # Save as HTML after full stream is complete
    if save_as_html:
        html_output = md_to_html(response)
        filename = f"{company_name.replace(' ', '_')}_pitch.html"
        with open(filename, "w", encoding="utf-8") as f:
            f.write(html_output)
        print(f"Pitch saved to: {filename}")


In [13]:
link_json = get_links("https://BlackRock.com")
homepage = link_json["homepage"]
links = link_json["links"]

stream_brochure("BlackRock", homepage, links)


# 📌 Company Name and Website
**BlackRock**  
[BlackRock.com](https://BlackRock.com)

![Company Logo](https://logo.clearbit.com/BlackRock.com)

---

## 🏢 Company Overview
BlackRock, Inc. is the world’s largest asset manager, with over $10 trillion in assets under management (AUM). Founded in 1988 and headquartered in New York City, BlackRock provides investment management, risk management, and advisory services to institutional and individual investors globally.

## ⚙️ Products & Services
BlackRock offers a diverse array of products including:
- **iShares ETFs:** A leader in exchange-traded funds, allowing investors easy diversification.
- **Mutual Funds:** Comprehensive options covering various asset classes.
- **Technology Solutions:** Includes Aladdin, a risk management platform that supports its investment strategies.

[Explore Products](https://www.blackrock.com/us/individual/products/investment-funds#/?productView=all&style=44341|44342)

## 💼 Business Model
BlackRock generates revenue via management fees on assets under management and performance fees, ensuring a steady income flow. The company’s fee structure benefits from market growth and increasing demand for passive investment strategies.

## 📈 Growth Strategy
- **Innovative Products:** Continuous development of sustainable investment funds, integrating ESG criteria.
- **Global Expansion:** Targeting emerging markets to capture new investor capital.
- **Digital Transformation:** Leveraging technology and data analytics to enhance client services.

## 🌱 ESG & Sustainability
BlackRock emphasizes sustainability as a core principle, integrating ESG factors into their investment process. This approach not only aligns with global sustainability goals but also captures the growing market preference for responsible investing.

[Learn More About Sustainability](https://www.blackrock.com/corporate/sustainability/our-approach-to-sustainability)

## 👥 Leadership
Under the helm of CEO Larry Fink, BlackRock's leadership team is distinguished by its expertise in finance, technology, and global markets. The firm promotes a culture of transparency and accountability, strengthening investor trust.

[Meet The Team](https://www.blackrock.com/corporate/about-us/investment-stewardship)

## 💰 Financial & Investor Highlights
- **Strong Revenue Growth:** AUM growth driven by market appreciation and new inflows.
- **Profit Margins:** Consistent margins with a 30% net profit margin.
- **Return on Equity (ROE):** Over 15% indicating efficient capital use.

## 🧠 Competitive Edge
BlackRock's competitive advantage lies in:
- **Scale and Market Leadership:** Extensive resources to scale innovation and drive down costs.
- **Robust Technology Platform:** Aladdin differentiates BlackRock's client services.
- **Brand Trust:** Recognized globally as a reliable asset manager, enhancing client loyalty.

## ✅ Final Recommendation
**BUY** 🟢  
With strong fundamentals, a commitment to sustainability, and a growth-oriented strategy, BlackRock is well-positioned to capitalize on the evolving asset management landscape. Investors can expect solid returns supported by its robust model and innovative offerings.


Pitch saved to: BlackRock_pitch.html
